## Barrel shifter (cyhdl library)

This variant uses mixed myIRL and cythonized myhdl code segments. Note that the procedural generation is quite a bit faster.

We import the barrelshifter library:

In [1]:
from library.barrelshifter import *

[Edit barrelshifter.py](library/barrelshifter.py)

Then we create a barrel shifter generator with data width 32 (power 5):

In [2]:
W_POWER = 5

TEST_VALUES = [
    (0xdead, 8, 0xdead << 8),
    (0x8f01, 15, 0x8f01 << 15),
]

b = BarrelShifterGenerator(W_POWER)

DEBUG CALLING CYTHONIZED CODE


Then we instance it first time in the test bench below. Note the instancing time on this first run.

In [3]:
from myirl.test.common_test import gen_osc
from myirl.emulation.myhdl import *

@block
def top_bs(b):
    clk = ClkSignal()
    ce = Signal(bool())
    val, result = [ Signal(intbv(0xaa00)[2 ** W_POWER:]) for i in range(2) ]
    s = Signal(intbv()[W_POWER:])
    
    inst = [
        b.barrel_shifter(clk, ce, val, s, result, False),
        gen_osc(clk, 2)
    ]


    @instance
    def stim():
        for item in TEST_VALUES:
            ce.next = False
            s.next = item[1]
            val.next = item[0]
            yield(clk.posedge)
            ce.next = True
            yield(clk.posedge)
            yield(clk.posedge)

            print(result)
            assert result == item[2]
            
        raise StopSimulation

    
    inst += [ stim ]
    return inst
 
@utils.timer
def test(b):    
    return top_bs(b)

design = test(b)

 Declare obj 'barrel_shifter' in context '(LIB: BarrelShifterGenerator 'bs')' 
Adding rotate to port dict (fallback)
 Declare obj 'shifter_stage' in context '(LIB: BarrelShifterGenerator 'bs')' 
Adding nmux to port dict (fallback)
Adding asr to port dict (fallback)
Adding rotate to port dict (fallback)
 Declare obj 'cshift' in context '(LIB: BarrelShifterGenerator 'bs')' 
Adding asr to port dict (fallback)
Adding rotate to port dict (fallback)
Adding WRAP to port dict (fallback)
Adding asr to port dict (fallback)
Adding rotate to port dict (fallback)
Adding WRAP to port dict (fallback)
Adding asr to port dict (fallback)
Adding rotate to port dict (fallback)
Adding WRAP to port dict (fallback)
Adding asr to port dict (fallback)
Adding rotate to port dict (fallback)
Adding WRAP to port dict (fallback)
Adding asr to port dict (fallback)
Adding rotate to port dict (fallback)
Adding WRAP to port dict (fallback)
Adding asr to port dict (fallback)
Adding rotate to port dict (fallback)
Adding 

Adding asr to port dict (fallback)
Adding rotate to port dict (fallback)
Adding WRAP to port dict (fallback)
Adding asr to port dict (fallback)
Adding rotate to port dict (fallback)
Adding WRAP to port dict (fallback)
Adding asr to port dict (fallback)
Adding rotate to port dict (fallback)
Adding WRAP to port dict (fallback)
Adding asr to port dict (fallback)
Adding rotate to port dict (fallback)
Adding WRAP to port dict (fallback)
Adding asr to port dict (fallback)
Adding rotate to port dict (fallback)
Adding WRAP to port dict (fallback)
Adding asr to port dict (fallback)
Adding rotate to port dict (fallback)
Adding WRAP to port dict (fallback)
Adding nmux to port dict (fallback)
Adding asr to port dict (fallback)
Adding rotate to port dict (fallback)
 Module bs: Existing instance shifter_stage, rename to shifter_stage_4 
Adding asr to port dict (fallback)
Adding rotate to port dict (fallback)
Adding WRAP to port dict (fallback)
Adding asr to port dict (fallback)
Adding rotate to port

Running the test bench again, the duration is a little less than on the first run, because this module was already compiled.

In [4]:
design = test(b)

Finished test in 0.0116 secs


Finally, the quick & dirty verification:

In [5]:
from myirl import targets
from myirl.test.common_test import run_ghdl

f = design.elab(targets.VHDL, elab_all = True)
f += b.elab(targets.VHDL)
# print(f)
run_ghdl(f, design, vcdfile = 'bs.vcd', debug = True)

 Elaborating component top_bs__BarrelShifterGenerator 
 Writing 'top_bs' to file /tmp/myirl_top_bs_w70jqicm/top_bs.vhdl 
 Creating library file /tmp/myirl_module_defs_qdcslcdu/module_defs.vhdl 
 Elaborating component shifter_stage__BarrelShifterGenerator_s32_s32_s1_16_s1_0_0 
 Writing 'shifter_stage_4' to file /tmp/myirl_bs_62y_j1cz/shifter_stage_4.vhdl 
 Elaborating component shifter_stage__BarrelShifterGenerator_s32_s32_s1_8_s1_0_0 
 Writing 'shifter_stage_3' to file /tmp/myirl_bs_62y_j1cz/shifter_stage_3.vhdl 
 Elaborating component shifter_stage__BarrelShifterGenerator_s32_s32_s1_4_s1_0_0 
 Writing 'shifter_stage_2' to file /tmp/myirl_bs_62y_j1cz/shifter_stage_2.vhdl 
 Elaborating component shifter_stage__BarrelShifterGenerator_s32_s32_s1_2_s1_0_0 
 Writing 'shifter_stage_1' to file /tmp/myirl_bs_62y_j1cz/shifter_stage_1.vhdl 
 Elaborating component cshift__BarrelShifterGenerator_s1_s1_s1_s1_s1_0_0_1 
 Writing 'cshift_1' to file /tmp/myirl_bs_62y_j1cz/cshift_1.vhdl 
 Elaborating co

/home/testing/.local/lib/python3.9/site-packages/myirl-0.0.0-py3.9-linux-x86_64.egg/myirl/kernel/components.py:179: UserWarning: Unspecified port I/O `msb` => IN
  base.warnings.warn("Unspecified port I/O `%s` => IN" % n)
/home/testing/.local/lib/python3.9/site-packages/myirl-0.0.0-py3.9-linux-x86_64.egg/myirl/kernel/components.py:179: UserWarning: Unspecified port I/O `sbit` => IN
  base.warnings.warn("Unspecified port I/O `%s` => IN" % n)


==== COSIM stderr ====
/tmp/myirl_module_defs_fycmvm5f/module_defs.vhdl:6:1:warning: package "module_defs" was also defined in file "/tmp/myirl_module_defs_qdcslcdu/module_defs.vhdl" [-Wlibrary]

==== COSIM stdout ====
0x00DEAD00
0x47808000
/tmp/myirl_top_bs_w70jqicm/top_bs.vhdl:70:9:@22ns:(assertion failure): Stop Simulation
/tmp/top_bs:error: assertion failed
in process .top_bs(myirl).stim
/tmp/top_bs:error: simulation failed



0

In [6]:
# !cat -n /tmp/barrel_shifter.vhdl